# REST API와 GraphQL API 비교

#### 환경 설정

In [1]:
import requests
import pandas as pd
import time

## Over fetching

모든 레시피의 id, title, difficultyLevel를 조회하는 경우를 살펴보자.

### REST API

In [2]:
recipe_list_url = 'http://localhost:8000/rest-api/recipe/'

start = time.time()
response = requests.get(recipe_list_url)
end = time.time()
print(end - start)

2.0384984016418457


In [3]:
recipe_list = response.json()
df = pd.DataFrame(recipe_list)
df.head()

,id,user,title,description,preparation_time,cooking_time,difficulty_level
0,1,2,Korean Bibimbap zz,Bibimbap is a signature Korean dish known for ...,30,25,Moderate
1,2,2,Kimchijeon,Kimchijeon is a savory Korean pancake made wit...,15,15,Easy
2,3,2,Dubu Jorim,Dubu Jorim is a flavorful Korean tofu dish coo...,15,20,Easy
3,4,3,계란 후라이,egg fry,0,3,쉬움
4,5,2,Scramble Egg,Egg,0,3,Easy


레시피의 user, description, preparation_time, cooking_time까지 조회되게 된다.

### GraphQL



In [4]:
recipe_query = """query MyQuery {
  allRecipes {
    id
    title
    difficultyLevel
  }
}"""

In [5]:
graphql_url = 'http://localhost:8000/graphql/'  


start = time.time()
response = requests.get(graphql_url, json={'query': recipe_query})
end = time.time()
print(end - start)

2.048164129257202


In [6]:
recipe_list = response.json()['data']['allRecipes']
df = pd.DataFrame(recipe_list)
df.head()

,id,title,difficultyLevel
0,1,Korean Bibimbap zz,Moderate
1,2,Kimchijeon,Easy
2,3,Dubu Jorim,Easy
3,4,계란 후라이,쉬움
4,5,Scramble Egg,Easy


## Under fetching

모든 레시피의 id, title, difficultyLevel와 레시피에 들어가는 재료의 정보를 조회하는 경우를 살펴보자.


### REST API

In [7]:
start = time.time()

In [8]:
recipe_list_url = 'http://localhost:8000/rest-api/recipe/'
response = requests.get(recipe_list_url)

recipe_list = response.json()
recipe_df = pd.DataFrame(recipe_list)
recipe_df.head()

,id,user,title,description,preparation_time,cooking_time,difficulty_level
0,1,2,Korean Bibimbap zz,Bibimbap is a signature Korean dish known for ...,30,25,Moderate
1,2,2,Kimchijeon,Kimchijeon is a savory Korean pancake made wit...,15,15,Easy
2,3,2,Dubu Jorim,Dubu Jorim is a flavorful Korean tofu dish coo...,15,20,Easy
3,4,3,계란 후라이,egg fry,0,3,쉬움
4,5,2,Scramble Egg,Egg,0,3,Easy


In [9]:
recipe_ingredient_list_url = 'http://localhost:8000/rest-api/recipe-ingredient/'
response = requests.get(recipe_ingredient_list_url)

recipe_ingredient_list = response.json()
recipe_ingredient_df = pd.DataFrame(recipe_ingredient_list)
recipe_ingredient_df.head()

,recipe,ingredient,quantity,unit
0,1,1,4.00,cups
1,1,2,200.00,grams
2,1,3,3.00,tablespoons
3,1,4,2.00,tablespoons
4,1,5,1.00,tablespoons


In [10]:
ingredient_list_url = 'http://localhost:8000/rest-api/ingredient/'
response = requests.get(ingredient_list_url)
ingredient_list = response.json()
ingredient_df = pd.DataFrame(ingredient_list)
ingredient_df.head()

,id,name
0,1,Cooked rice
1,2,Beef
2,3,Soy sauce
3,4,Sesame oil
4,5,Sugar


In [11]:
cols = ['id', 'title', 'difficulty_level']
recipe_df = recipe_df[cols]

In [12]:
df = (
    recipe_ingredient_df
    .merge(recipe_df, left_on='recipe', right_on='id', how='left')
    .drop(columns=['id'])
    .merge(ingredient_df, left_on='ingredient', right_on='id', how='left')
    .drop(columns=['id', 'ingredient'])
    .rename(columns={'name': 'ingredient_name', 'recipe': 'id'})
)
df.head()

,id,quantity,unit,title,difficulty_level,ingredient_name
0,1,4.00,cups,Korean Bibimbap zz,Moderate,Cooked rice
1,1,200.00,grams,Korean Bibimbap zz,Moderate,Beef
2,1,3.00,tablespoons,Korean Bibimbap zz,Moderate,Soy sauce
3,1,2.00,tablespoons,Korean Bibimbap zz,Moderate,Sesame oil
4,1,1.00,tablespoons,Korean Bibimbap zz,Moderate,Sugar


In [13]:
end = time.time()
print(end - start)

6.246707439422607


필요한 정보를 전부 가져오기 위해 여러 번의 요청을 보내야 한다. 

### GraphQL



In [14]:
recipe_ingredient_query = """query MyQuery {
  allRecipes {
    id
    title
    difficultyLevel
    recipeingredientSet {
      ingredient {
        name
      }
      quantity
      unit
    }
  }
}"""

In [15]:
start = time.time()
response = requests.get(graphql_url, json={'query': recipe_ingredient_query})
recipe_list = response.json()['data']['allRecipes']
end = time.time()
print(end - start)

2.2221717834472656


In [16]:
df = pd.json_normalize(
    recipe_list,
    record_path='recipeingredientSet',
    meta=['id', 'title', 'difficultyLevel']
)
df.head()

,quantity,unit,ingredient.name,id,title,difficultyLevel
0,4.00,cups,Cooked rice,1,Korean Bibimbap zz,Moderate
1,200.00,grams,Beef,1,Korean Bibimbap zz,Moderate
2,3.00,tablespoons,Soy sauce,1,Korean Bibimbap zz,Moderate
3,2.00,tablespoons,Sesame oil,1,Korean Bibimbap zz,Moderate
4,1.00,tablespoons,Sugar,1,Korean Bibimbap zz,Moderate


한번의 요청으로 필요한 데이터를 전부 가져온다.